In [1]:
import pyspark

import os
from datetime import date
import functools
from IPython.core.display import display, HTML
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import count, isnan, mean, col, countDistinct, format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format, when,datediff, months_between

import re 
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("vaccine").getOrCreate()

# Read data

In [3]:
df = spark.read.csv('/Volumes/My Passport/PAMEpi_datalake/raw_data_covid19_version-21-12-07/data-datasus_vacinacao_brasil/part-00000-a9fa6c84-7d08-410d-bdc8-fd5f338134c4-c000.csv', sep=';', header = True, encoding="utf-8", inferSchema = True)

In [19]:
df.count()

303417142

In [7]:
df.limit(5).toPandas() 

,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,...,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem,vacina_numdose
0,10fd6162-15a0-4c69-b5ef-6d45d901b8f4-i0b0,4cc05cb74a857154d305a6ab200f73da1338058e13bce8...,71,1949-12-30,F,3,PARDA,355030,10,SAO PAULO,...,SINOVAC/BUTANTAN,None,2021-03-26,1Âª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,VACIVIDA,2021-03-26 04:00:00,18262,1
1,1c6d1b97-7bdc-40d2-acd6-f0d42d825725-i0b0,c6e434ee9e9c7dfce92b04d08a553bb6ddecf328a1235d...,82,1939-01-16,F,3,PARDA,314330,10,MONTES CLAROS,...,SINOVAC/BUTANTAN,Organization/61189445000156,2021-03-03,1Âª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,Novo PNI,2021-03-03 13:48:04,16341,1
2,5f5d27de-ff38-40d2-814d-b2d6f916bda4-i0b0,c1dc811e8a88c185f0644c5dfb8446e219672699c3f428...,78,1942-10-09,F,1,BRANCA,352230,10,ITAPETININGA,...,ASTRAZENECA/FIOCRUZ,None,2021-03-04,1Âª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,VACIVIDA,2021-03-04 04:03:00,18262,1
3,5465ae80-3c59-49bf-b78e-61247d400005-i0b0,13824e8fd9bae9ab45e085aa0906858aa2ec4afaf97815...,74,1946-10-11,F,4,AMARELA,270760,10,QUEBRANGULO,...,SINOVAC/BUTANTAN,Organization/61189445000156,2021-03-24,1Âª Dose,86,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,Novo PNI,2021-03-26 01:28:09,16341,1
4,c6153b6f-b280-4e4d-be1b-83715d10847b-i0b0,c10fffd9c3008c2f7b97d157b3cc05d9803b67a46594f2...,64,1956-07-19,F,3,PARDA,130260,10,MANAUS,...,ASTRAZENECA/FIOCRUZ,None,2021-03-11,1Âª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,SMV,2021-03-13 13:24:30,18183,1


In [9]:
df.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: integer (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_racacor_codigo: integer (nullable = true)
 |-- paciente_racacor_valor: string (nullable = true)
 |-- paciente_endereco_coibgemunicipio: integer (nullable = true)
 |-- paciente_endereco_copais: integer (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: integer (nullable = true)
 |-- paciente_nacionalidade_enumnacionalidade: string (nullable = true)
 |-- estabelecimento_valor: integer (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: integer (null

# Date range of the dataset

In [5]:
#from pyspark.sql.functions import to_timestamp


#df.select(to_timestamp(df.vacina_dataaplicacao, 'yyyy-MM-dd HH:mm:ss').alias('dt')).collect()

In [6]:
df2=df.select(countDistinct("paciente_id"))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o33.select

In [ ]:
df2.show()

# Basic counts 

In [20]:
df.groupBy(['paciente_endereco_uf']).count().show(50,truncate=False)

+--------------------+--------+
|paciente_endereco_uf|count   |
+--------------------+--------+
|SC                  |11105409|
|RO                  |2326017 |
|PI                  |4766396 |
|AM                  |4743694 |
|RR                  |656114  |
|GO                  |9607323 |
|null                |1502763 |
|TO                  |1995138 |
|MT                  |4733395 |
|SP                  |73272118|
|PB                  |6044263 |
|ES                  |6167192 |
|RS                  |18096279|
|XX                  |49730   |
|MS                  |4345740 |
|AL                  |4301517 |
|MG                  |31300844|
|PA                  |9138138 |
|BA                  |19368987|
|SE                  |2924146 |
|PE                  |12822632|
|CE                  |12018539|
|RN                  |5109496 |
|RJ                  |25338431|
|MA                  |7872899 |
|AC                  |1033155 |
|DF                  |4303969 |
|PR                  |17638401|
|AP     

In [2]:
#df.groupby('vacina_nome').count().show(truncate=False)

In [ ]:
df.groupby('vacina_nome').count().show(truncate=False)

In [5]:
df.groupby('vacina_numdose').count().show(truncate=False)

+--------------+---------+
|vacina_numdose|count    |
+--------------+---------+
|27            |1        |
|1             |157128841|
|6             |33169    |
|3             |460152   |
|37            |648165   |
|9             |229783   |
|8             |4354632  |
|39            |2        |
|10            |5        |
|38            |14647055 |
|32            |765      |
|33            |841      |
|11            |2        |
|2             |125912351|
|36            |1378     |
+--------------+---------+



In [6]:
df.groupby('vacina_descricao_dose').count().show(truncate=False)

+------------------------------+---------+
|vacina_descricao_dose         |count    |
+------------------------------+---------+
|Reforço                       |14647055 |
|Revacinação                   |5        |
|2ª Dose                       |125912351|
|1ª Dose                       |157128841|
|Única                         |229783   |
|1º Reforço                    |33169    |
|Dose Inicial                  |1378     |
|Tratamento com dezessete doses|1        |
|3ª Dose                       |460152   |
|Dose                          |4354632  |
|Dose Adicional                |648165   |
|3º Reforço                    |2        |
|1ª Dose Revacinação           |765      |
|Tratamento com uma dose       |2        |
|2ª Dose Revacinação           |841      |
+------------------------------+---------+



In [ ]:
df.groupby(['vacina_nome','vacina_descricao_dose']).count().show(1000,truncate=False)

['paciente_idade','paciente_enumsexobiologico',
 'paciente_racacor_codigo', 'paciente_racacor_valor',
 'paciente_endereco_coibgemunicipio', 'paciente_endereco_nmmunicipio',
 'paciente_endereco_uf', 'vacina_grupoatendimento_codigo',
 'vacina_grupoatendimento_nome', 'vacina_categoria_codigo',
 'vacina_categoria_nome', 'vacina_dataaplicacao','vacina_descricao_dose',
 'vacina_codigo','vacina_nome','vacina_numdose']

In [8]:
df1 = df.groupby(['paciente_endereco_uf','paciente_endereco_nmmunicipio','paciente_endereco_coibgemunicipio','vacina_nome','vacina_codigo']).count().coalesce(1).write.format("csv").save("/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/Vaccine_covid/type_vac_muni.csv")#.show(truncate=False)

# Harmonisations

# Plots

# Global data

In [ ]:
https://covid19.who.int/who-data/vaccination-data.csv